# Search Engine

This lab is about starting from scratch, and hence having to structure the code yourself. A specification for the program you are to implement is given below - do pay attention as misunderstandings may cost you marks! Being precise is a necessary skill for a programmer. To give an executive summary, you are to code a search engine for recipes. A data set has been provided. The search engine is to be pretty basic, returning all recipes that contain all of the provided keywords. However, the user can choose from a number of orderings depending on their food preferences, which you need to support.

## Marking and submission

These lab exercises are marked, and contribute to your final grade. This lab exercise has 20 marks to earn, equivalent to 12% of your final grade.

Please submit your completed workbook to the auto marker before 2021-11-14 20:00 GMT. The workbook you submit must be an .ipynb file, which is saved into the directory you're running Jupyter; alternatively you can download it from the menu above using `File -> Download As -> Notebook (.ipynb)`. Remember to save your work regularly (`Save and checkpoint` in the `File` menu, the icon of a floppy disk, or `Ctrl-S`). It is wise to verify it runs to completion with _Restart & Run All_ before submission.

You must comply with the universities plagiarism guidelines: http://www.bath.ac.uk/library/help/infoguides/plagiarism.html

## Specification

The system must provide a function ``search``, with the following specification:
```
def search(query, ordering = 'normal', count = 10):
  ...
```

It `print`s out the results of the search, subject to the following rules:
1. It selects from the set of all recipes that contain __all__ of the words in the query (the positions/order of the words in the recipe are to be ignored).
2. It orders them based on the provided ordering (a string, meaning defined below).
3. It `print`s the top `count` matches only, preserving the order from best to worst. Must `print` just their title, one per line. Must be less than `count` if the search is specific enough that less than `count` recipes match.

As an aside, do not worry about memory usage. If duplicating some data can make your code faster/neater then feel free.



### Data set

A file, `recipes.json` is provided, containing 17K recipes. It can be parsed into a Python data structure using the [`json`](https://docs.python.org/3/library/json.html) module. It is a list, where each recipe is a dictionary containing various keys:
* `title` : Name of recipe; you can assume these are unique
* `categories` : A list of tags assigned to the recipe
* `ingredients` : What is in it, as a list
* `directions` : List of steps to make the recipe
* `rating` : A rating, out of 5, of how good it is
* `calories` : How many calories it has
* `protein` : How much protein is in it
* `fat` : How much fat is in it

Note that the data set was obtained via web scrapping and hence is noisy - every key except for `title` is missing from at least one recipe. Your code will need to cope with this.

You will probably want to explore the data before starting, so you have an idea of what your code has to deal with.

Data set came from https://www.kaggle.com/hugodarwood/epirecipes/version/2 though note it has been cleaned it up, by deleting duplicates and removing the really dodgy entries.



### Search

The search should check the following parts of the recipe (see data set description below):
* `title`
* `categories`
* `ingredients`
* `directions`

For instance, given the query "banana cheese" you would expect "Banana Layer Cake with Cream Cheese Frosting" in the results. Note that case is to be ignored ("banana" matches "Banana") and the words __do not__ have to be next to one another, in the same order as the search query or even in the same part of the recipe ("cheese" could appear in the title and "banana" in the ingredients). However, all words in the search query __must__ appear somewhere.



### Tokenisation

This is the term for breaking a sentence into each individual word (token). Traditionally done using regular expressions, and Python does have the `re` module, but there is no need to do that here (`re` can be quite fiddly). For matching words your tokenisation must follow the following steps:
1. Convert all punctuation and digits into spaces. For punctuation use the set in [`string.punctuation`](https://docs.python.org/3/library/string.html#string.punctuation), for digits [`string.digits`](https://docs.python.org/3/library/string.html#string.digits). You may find [`translate()`](https://docs.python.org/3/library/stdtypes.html#str.translate) interesting!
2. [`split()`](https://docs.python.org/3/library/stdtypes.html#str.split) to extract individual tokens.
3. Ignore any token that is less than $3$ characters long.
4. Make tokens lowercase.

When matching words for search (above) or ordering (below) it's only a match if you match an entire token. There are many scenarios where this simple approach will fail, but it's good enough for this exercise. The auto marker will be checking the above is followed! When doing a search the code should ignore terms in the search string that fail the above requirements.



### Ordering

There are three ordering modes to select from, each indicated by passing a string to the `search` function:
* `normal` - Based simply on the number of times the search terms appear in the recipe. A score is calculated and the order is highest to lowest. The score sums the following terms (repeated words are counted multiple times, i.e. "cheese cheese cheese" is $3$ matches to "cheese"):
    * $8 \times$ Number of times a query word appears in the title
    * $4 \times$ Number of times a query word appears in the categories
    * $2 \times$ Number of times a query word appears in the ingredients
    * $1 \times$ Number of times a query word appears in the directions
    * The `rating` of the recipe (if not available assume $0$)

* `simple` - Tries to minimise the complexity of the recipe, for someone who is in a rush. Orders to minimise the number of ingredients multiplied by the numbers of steps in the directions.

* `healthy` - Order from lowest to highest by this cost function:
$$\frac{|\texttt{calories} - 510n|}{510} + 2\frac{|\texttt{protein} - 18n|}{18} + 4\frac{|\texttt{fat} - 150n|}{150}$$
Where $n \in \mathbb{N}^+$ is selected to minimise the cost ($n$ is a positive integer and $n=0$ is not allowed). This can be understood in terms of the numbers $510$, $18$ and $150$ being a third of the recommended daily intake (three meals per day) for an average person, and $n$ being the number of whole meals the person gets out of cooking/making the recipe. So this tries to select recipes that neatly divide into a set of meals that are the right amount to consume for a healthy, balanced diet. Try not to overthink the optimisation of $n$, as it's really quite simple to do!

To clarify the use of the ordering string, to get something healthy that contains cheese you might call `search('cheese', 'healthy')`. In the case of a recipe that is missing a key in its dictionary the rules are different for each search mode:
* `normal` - Consider a missing entry in the recipe (e.g. no `ingredients` are provided) to simply mean that entry can't match any search words (because it has none!), but the item is still eligible for inclusion in the results, assuming it can match the search with a different entry.
* `simple` - If a recipe is missing either `ingredients` or `directions` it is dropped from such a search result. Because the data is messy if either of these lists is of length $1$ it should be assumed that the list extraction has failed and the recipe is to also be dropped from the search results.
* `healthy` - If any of `calories`, `protein` or `fat` is missing the recipe should be dropped from the result.



### Extra

You may find the [_inverted index_](https://en.wikipedia.org/wiki/Inverted_index) interesting. It's a data structure used by search engines. For each word a user may search for this contains a list of all documents (recipes) that contain the word. This may take a little effort to understand, but the resulting code will be both faster and neater.

## Advice

* Don't just start coding: Make a plan and work out what you intend to do.
* Think about structure, as messy code leads to mistakes.
* Plan your data structures. Don't be afraid to use sheets of paper if that works for you!
* Don't duplicate code, put it in a function/method instead.

* Divide and conquer. Break the system into parts that can implemented with minimal dependency on the rest. Functions or OOP are both suitable for doing this.
* Test early. Verify the individual parts work before trying to combine them. Factor this into the order you implement the parts of the system - don't implement something you are going to struggle to test before implementing, and verifying, other parts.
* Do not try and do a 'big bang', where you get it all working at once. Instead, get it working with features missing, then add those features in, one at a time.

* Keep things as simple as possible. Avoid long functions/methods.
* Include comments, as a form of planning and for your own sanity!
* Regularly reset the kernel and rerun the entire workbook. It is very easy to break something but not notice, because the correct version remains in memory.

## Marks
* __20 marks__: For many different _unit tests_ that will be run on `search`. They will cover all of the details in the above specification!
    * One test checks that it's faster than $0.1$ seconds on average (on the computer which runs the auto marker, which is quite fast) to do a search, so try not to be too inefficient (it ignores any time your notebook spends building data structures to be used by `search`). Note that the validation implementation comes in at $0.001$ seconds per search (after $5.5$ seconds of preparation), so this is very achievable!
    * You may want to look into Python's `set()` object, as it is useful for _one_ of the possible ways to implement this.
    * There will be sorting. The [Sorting how to](https://docs.python.org/3/howto/sorting.html) may help.
    * The auto marker does give some feedback, and you can run it as many times as you want. Don't be afraid to test an incomplete or semi-broken version of your code if stuck or unsure. It may help!
    * The validation implementation is 104 lines of code split over 5 cells (including white space for clarity and comments). Coded by someone who probably has much more experience than you, so you shouldn't aim to match this, but it's a good clue: If you find yourself at 500 lines you may want to stop and think some more! (line count does not include testing code, which is about the same amount again)

In [144]:
#importing the required libraries
import json
import string
import numpy
import time

In [152]:
#loading the file in f
#F = open('/Users/souravpanda/Desktop/university of bath/study/software technology for data srecipes.json')
data = json.load(open('recipes.json'))


In [153]:
#G = open('/Users/souravpanda/Desktop/university of bath/study/software technology for data science/lab/recipes.json')
token_data = json.load(open('recipes.json'))


In [154]:
s_data = json.load(open('recipes.json'))

In [155]:
s_token_data = json.load(open('recipes.json'))

In [62]:
#s is a word, i is the char a is the new form of word
def remove_punc(s):
    a=''
    lst = []
    for i in s:
        if i in string.punctuation:
            a+=' '
        elif i in string.digits:
            pass
        else:
            a+=i.lower()
    lst = a.split()
    
    for i in lst:
        if len(i)<3:
            lst.remove(i)
    return lst

In [151]:
for i in range(len(token_data)):
    token_data[i]['title']=remove_punc(token_data[i]['title'])

    


In [157]:
#Tokenising cont for rest
tok_lst = ['categories','ingredients','directions']
for i in range(len(token_data)):
    for s in tok_lst:
    
        a= []
        for j in token_data[i][s]:
            a+=(remove_punc(j))
        token_data[i][s]=a

In [150]:
for j in range(len(s_data)): 
    i = s_data[j] 
    if 'categories' not in i.keys(): 
        #print(j,'c')
        i['categories'] = []
    if 'ingredients' not in i.keys():
        #print(j,'i')
        i['ingredients'] = []
    if 'directions' not in i.keys():
        #print(j,'d')
        i['directions'] = []
    if 'rating' not in i.keys():
        #print(j,'r')
        i['rating'] = 0
    if 'calories' not in i.keys():
        #print(j,'d')
        i['calories'] = 0
    if 'protein' not in i.keys():
        #print(j,'d')
        i['protein'] = 0
    if 'fat' not in i.keys():
        #print(j,'d')
        i['fat'] = 0

In [66]:
for i in range(len(token_data)):
    token_data[i]['all_tok']= token_data[i]['title']+token_data[i]['categories']+token_data[i]['ingredients']+token_data[i]['directions']
    

In [65]:
for i in range(len(s_token_data)):
    s_token_data[i]['all_tok']= s_token_data[i]['title']+s_token_data[i]['categories']+s_token_data[i]['ingredients']+s_token_data[i]['directions']

In [10]:
for i in range(len(s_data)):
    s_data[i]['hscore'] = abs((s_data[i]['calories']-510)/510)+abs((s_data[i]['protein']-18)/9)+abs((2/75)*(s_data[i]['fat']-150)) 
    

In [11]:
for i in range(len(s_token_data)):
    #print(i)
    s_data[i]['ilen'] = len(s_data[i]['ingredients'])
    s_data[i]['dlen'] = len(s_data[i]['directions'])
    s_data[i]['score'] = ( s_data[i]['ilen']*s_data[i]['dlen'])
    s_token_data[i]['i'] = i

In [24]:
def simple_search(query,count):
    s_result = []
    s = query
    for i in range(len(s_token_data)):
        #print(set(s)<= set(token_data[0]['title']))
        if (s_data[i]['ilen']==1 or s_data[i]['ilen']==0 or s_data[i]['dlen']==1 or s_data[i]['dlen']==0):
            pass
        else:
        
            if set(s) <= set(s_token_data[i]['all_tok']):
                s_result.append(s_data[i])
    out1 = sorted(s_result, key=lambda i: i['score'],)[:count]
    return out1

In [25]:
def healthy_search(query,count): 
    h_result = []
    s = query
    for i in range(len(s_token_data)):
        if (s_data[i]['calories']==0 or s_data[i]['protein']==0 or s_data[i]['fat']==0):
            #print('hi')
            pass
        else:
            #print('j')
            if set(s) <= set(s_token_data[i]['all_tok']):
                #print('k')
                h_result.append(s_data[i])

    out = sorted(h_result, key=lambda i: i['hscore'],)[:count]
    return out

In [156]:
#Adding the miising key also giving them apropiate values.
for j in range(len(token_data)): 
    i = token_data[j] 
    if 'categories' not in i.keys(): 
        #print(j,'c')
        i['categories'] = ['Dose_not_eXist']
    if 'ingredients' not in i.keys():
        #print(j,'i')
        i['ingredients'] = ['Dose_not_eXist']
    if 'directions' not in i.keys():
        #print(j,'d')
        i['directions'] = ['Dose_not_eXist']
    if 'rating' not in i.keys():
        #print(j,'r')
        i['rating'] = -1
    if 'calories' not in i.keys():
        #print(j,'d')
        i['calories'] = -1
    if 'protein' not in i.keys():
        #print(j,'d')
        i['protein'] = -1
    if 'fat' not in i.keys():
        #print(j,'d')
        i['fat'] = -1
    

In [160]:
#Tokenising the the title
for i in range(len(token_data)):
    token_data[i]['title']=remove_punc(token_data[i]['title'])

In [16]:
#Tokenising cont for rest
tok_lst = ['categories','ingredients','directions']
for i in range(len(token_data)):
    for s in tok_lst:
    
        a= []
        for j in token_data[i][s]:
            a+=(remove_punc(j))
        token_data[i][s]=a

        
    

In [161]:
token_data[1]

{'title': ['blanketed', 'eggplant'],
 'categories': ['tomato',
  'vegetable',
  'appetizer',
  'side',
  'vegetarian',
  'eggplant',
  'pan',
  'fry',
  'vegan',
  'bon',
  'appétit'],
 'ingredients': ['small',
  'japanese',
  'eggplants',
  'peeled',
  'large',
  'fresh',
  'mint',
  'leaves',
  'large',
  'garlic',
  'cloves',
  'slivered',
  'flattened',
  'cups',
  'olive',
  'oil',
  'for',
  'deep',
  'frying',
  'pounds',
  'tomatoes',
  'tablespoons',
  'extra',
  'virgin',
  'olive',
  'oil',
  'medium',
  'onion',
  'chopped',
  'fresh',
  'basil',
  'leaves',
  'tablespoon',
  'dried',
  'oregano',
  'tablespoons',
  'drained',
  'capers'],
 'directions': ['place',
  'eggplants',
  'double',
  'thickness',
  'paper',
  'towels',
  'salt',
  'generously',
  'let',
  'stand',
  'hour',
  'pat',
  'dry',
  'with',
  'paper',
  'towels',
  'cut',
  'deep',
  'incisions',
  'each',
  'eggplant',
  'using',
  'tip',
  'knife',
  'push',
  'mint',
  'leaf',
  'and',
  'garlic',
  '

In [120]:
search_keys = [*data[1].keys()][:4]
cal_keys = [*data[1].keys()][4:]


In [162]:
ii = []
for i in range(len(data)):
    score = {}
    for j in data[i]:
        if j == search_keys[0]:
            for w in token_data[i][j]:
                if w in score:
                    score[w]+=8
                else:
                    score[w]=8
        if j == search_keys[1]:
            for w in token_data[i][j]:
                if w in score:
                    score[w]+=4
                else:
                    score[w]=4
        if j == search_keys[2]:
            for w in token_data[i][j]:
                if w in score:
                    score[w]+=2
                else:
                    score[w]=2
        if j == search_keys[3]:
            for w in token_data[i][j]:
                if w in score:
                    score[w]+=1
                else:
                    score[w]=1
        else:
            pass
    score['index'] = i
    ii.append(score)
        
        

In [166]:
ii[0]

{'adult': 8,
 'pimiento': 8,
 'cheese': 13,
 'vegetable': 4,
 'cook': 4,
 'vegetarian': 4,
 'quick': 4,
 'easy': 4,
 'cheddar': 7,
 'hot': 4,
 'pepper': 5,
 'winter': 4,
 'gourmet': 4,
 'alabama': 4,
 'large': 3,
 'garlic': 4,
 'cloves': 2,
 'ounce': 2,
 'jar': 3,
 'diced': 2,
 'pimientos': 3,
 'cups': 2,
 'coarsely': 2,
 'grated': 2,
 'sharp': 2,
 'preferably': 2,
 'english': 2,
 'canadian': 2,
 'vermont': 2,
 'about': 2,
 'ounces': 2,
 'cup': 2,
 'mayonnaise': 3,
 'crackers': 2,
 'toasted': 2,
 'baguette': 2,
 'slices': 2,
 'crudités': 2,
 'force': 1,
 'through': 1,
 'press': 1,
 'into': 1,
 'bowl': 1,
 'and': 4,
 'stir': 2,
 'with': 3,
 'liquid': 1,
 'add': 1,
 'toss': 1,
 'mixture': 1,
 'combine': 1,
 'well': 1,
 'taste': 1,
 'season': 1,
 'freshly': 1,
 'ground': 1,
 'black': 1,
 'spread': 3,
 'may': 1,
 'made': 1,
 'day': 1,
 'ahead': 1,
 'chilled': 1,
 'covered': 1,
 'bring': 1,
 'room': 1,
 'temperature': 1,
 'before': 1,
 'serving': 1,
 'serve': 1,
 'accompaniments': 1,
 'inde

['rating', 'calories', 'protein', 'fat']

In [72]:
def search(query,ordering,count):
    s = remove_punc(query)
    
    if ordering.lower() =='normal':
        
        indx=[]
        result =[]
        #lst_key = ['title', 'categories', 'ingredients', 'directions']
        for i in range(len(token_data)):
            #print(set(s)<= set(token_data[0]['title']))
            if set(s) <= set(token_data[i]['all_tok']):
                indx.append(i)
                

    #Score ------------------------------------------------------------    
        score_lst = []
        for i in range(len(result)):
            score=0
            for w in s:
                for key in lst_key:
                    if w in result[i]['score'][key]:
                        #print(w,key,result[i]['score'][key][w])
                        score +=result[i]['score'][key][w]
            score+=result[i]['rating']
            score_lst.append(score)
    #top count no of score
        indx_score = sorted(range(len(score_lst)), key=lambda i: score_lst[i], reverse = True)[:count]

    #printing the results
        for f in indx_score:
            print(data[indx[f]]['title'])

    #SEARCH SIMPLE---------------------------------------------
    if ordering.lower() =='simple':
        
        si = simple_search(s,count)
        for i in si:
            print(i['title'])
     #SEARCH HEALTHY---------------------------------------------
    if ordering.lower()=='healthy':
        
        h=healthy_search(s,count)
        for i in h:
            print(i['title'])
        

In [167]:
a = time.time()
search('bread','normal',10)
b=time.time()

KeyError: 'all_tok'

In [74]:
all()

0.27873682975769043